# D3 in Jupyter

This notebook is a template for generating D3 visualizations in a Jupyter notebook. Jupyter notebooks are the best visualization format and D3 is the best visualization grammer: [por qué no los dos](https://www.youtube.com/watch?v=vqgSO8_cRio)? A quick adaptation of [this](https://github.com/cmoscardi/embedded_d3_example/blob/master/Embedded_D3.ipynb) (discovery not my own, see [here](http://blog.thedataincubator.com/2015/08/embedding-d3-in-an-ipython-notebook/)).

## CSS

In [58]:
%%html

<style>
div.bar {
    display: inline-block;
    width: 20px;
    height: 75px;
    background-color: teal;
}
</style>

## JavaScript

In [59]:
%%javascript

require.config({
    paths: {
        d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3'
    }
});

<IPython.core.display.Javascript object>

Simple SIGINT.

In [99]:
%%javascript
require(['d3'], function(d3){
    /*
        Jupyter notebooks save their state between runtimes
        so the chart needs to be explicitly destroyed and
        recreated every time the code is run.
    */
    $("#chart").remove();
    element.append("<div id='chart'></div>");

    
    var dataset = [ 5, 10, 15, 20, 25 ];
    var chart = d3.select("#chart")
    chart.selectAll("div").data(dataset).enter().append("div")
        .attr({
            "class": "bar"
        })
        .style({
            "height": function(d) {
                console.log(d);
                return d * 2.5 + "px";
        },
        "width": "20",
        "margin-right": "2px"
    });
});

<IPython.core.display.Javascript object>

Circles (testing CSS styles).

In [102]:
%%javascript
require(['d3'], function(d3){
    /*
        Jupyter notebooks save their state between runtimes
        so the chart needs to be explicitly destroyed and
        recreated every time the code is run.
    */
    $("#chart").remove();
    element.append("<div id='chart'></div>");

    
    var dataset = [ 5, 10, 15, 20, 25 ];
    var svg = d3.select("#chart").append("svg")
    svg.attr({
        "width": 750,
        "height": 250
    });
    svg.selectAll("circle").data(dataset).enter().append("circle").attr({
        "fill": "yellow",
        "stroke": "orange",
        "stroke-width": 3,
        "r": function(d) { return d; },
        "cy": 125,
        "cx": function(d, i) { return 250 + i*50 + i * d; }
    });
});

<IPython.core.display.Javascript object>

A proper bar graph, but pointed the wrong way and not scaled against the container.

In [118]:
%%javascript
require(['d3'], function(d3){
    /*
        Jupyter notebooks save their state between runtimes
        so the chart needs to be explicitly destroyed and
        recreated every time the code is run.        
    */
    $("#chart").remove();
    element.append("<div id='chart'></div>");

    
    var dataset = [ 5, 10, 13, 19, 21, 25, 22, 18, 15, 13,
                    11, 12, 15, 20, 18, 17, 16, 18, 23, 25 ];
    var svg = d3.select("#chart").append("svg")
    var w = 500
    var h = 250
    svg.attr({
        "width": w,
        "height": h
    });
    svg.selectAll("rect").data(dataset).enter().append("rect").attr({
        "fill": "black",
        "width": 20,
        "height": function(d) { return d; },
        "x": function(d, i) { return i * 21; }
    });
});

<IPython.core.display.Javascript object>

A proper bar graph, pointed the right way, centered vertically and horizontally, hand-scaled both vertically and horizontally.

The SVG origin `(0,0)` point as at the top-left corner, so you have to reverse `x` to get things pointed the right way. Since `SVG` spec doesn't allow negative coordinates this means we have to calculate the headspace and move the `x` that far down...not that well thought out.

Centering is best handled using external HTML, as here. `margin:auto; display:block;` doesn't seem to work, but `text-alight:center;` does. How sordid `CSS/HTML` centering is! I want my `<center>` back.

In [176]:
%%javascript
require(['d3'], function(d3){
    /*
        Jupyter notebooks save their state between runtimes
        so the chart needs to be explicitly destroyed and
        recreated every time the code is run.        
    */
    $("#chart").remove();
    element.append("<div id='chart' style='margin:auto; text-align:center;'></div>");
//     element.append("<div id='chart' style='margin:auto; diplay:block'></div>");
    
    
    var dataset = [ 5, 10, 13, 19, 21, 25, 22, 18, 15, 13,
                    11, 12, 15, 20, 18, 17, 16, 18, 23, 25 ];
    var svg = d3.select("#chart").append("svg");
    var w = 800;
    var h = 250;
    var bar_padding = 1;
    // "..." is the spread operator. It unpacks arrays?
    // You can't just call max on the array variable...
    console.log(Math.max(...dataset))
    // Must also compensate for padding in the below.
    var bar_width = w / dataset.length - bar_padding;
    var max_height = h
    var scaled_unit = h / Math.max(...dataset)
    svg.attr({
        "width": w,
        "height": h,
        /*
            The below is an SVG property which forces elements inside to conform to the nearest whole digit.
            Because a half a pixel is rendered as a blur, this forces sharpness.
        */
        "shape-rendering": "crispEdges",
    });
    svg.selectAll("rect").data(dataset).enter().append("rect").attr({
        "fill": "black",
        "width": bar_width,
        "height": function(d) { return d * scaled_unit; },
        "x": function(d, i) { return (bar_width + 1) * i; },
        /* 
            SVG does not allow negative coordinates so we have to "grow from above".
            ...wow this is crude. "Standards" haha damn.
        */
        "y": function(d) { return (Math.max(...dataset) - d) * scaled_unit },
    });
});

<IPython.core.display.Javascript object>

Add a dead-simple RBG-based color map. Create labels and center them within the graph.

In [216]:
%%javascript
require(['d3'], function(d3){
    /*
        Jupyter notebooks save their state between runtimes
        so the chart needs to be explicitly destroyed and
        recreated every time the code is run.        
    */
    $("#chart").remove();
    element.append("<div id='chart' style='margin:auto; text-align:center;'></div>");
//     element.append("<div id='chart' style='margin:auto; diplay:block'></div>");
    
    
    var dataset = [ 5, 10, 13, 19, 21, 25, 22, 18, 15, 13,
                    11, 12, 15, 20, 18, 17, 16, 18, 23, 25 ];
    var svg = d3.select("#chart").append("svg");
    var w = 800;
    var h = 250;
    var bar_padding = 1;
    // "..." is the spread operator. It unpacks arrays?
    // You can't just call max on the array variable...
    console.log(Math.max(...dataset))
    // Must also compensate for padding in the below.
    var bar_width = w / dataset.length - bar_padding;
    var max_height = h
    var scaled_unit = h / Math.max(...dataset)
    svg.attr({
        "width": w,
        "height": h,
        /*
            The below is an SVG property which forces elements inside to conform to the nearest whole digit.
            Because a half a pixel is rendered as a blur, this forces sharpness.
        */
        "shape-rendering": "crispEdges",
    });
    svg.selectAll("rect").data(dataset).enter().append("rect").attr({
        "fill": "black",
        "width": bar_width,
        "height": function(d) { return d * scaled_unit; },
        "x": function(d, i) { return (bar_width + 1) * i; },
        /* 
            SVG does not allow negative coordinates so we have to "grow from above".
            ...wow this is crude. "Standards" haha damn.
        */
        "y": function(d) { return (Math.max(...dataset) - d) * scaled_unit },
        "fill": function(d) { return "rgb(0, 0, " + (d * 10) + ")"; }
    });
    svg.selectAll("text")
        .data(dataset)
        .enter()
        .append("text")
        .text(function(d) { return d; })
        .attr({
            "fill": "white",
            "x": function(d, i) { return (bar_width + 1) * i + bar_width / 4 + 1; },
            "y": function(d) { return (Math.max(...dataset) - d) * scaled_unit + 20 },
        });
});

<IPython.core.display.Javascript object>

An SVG trick to get the labels properly centered: use `text-anchor` to move the centroid from the left-bottom edge to the bottom-center one using `middle`.

In [232]:
%%javascript
require(['d3'], function(d3){
    /*
        Jupyter notebooks save their state between runtimes
        so the chart needs to be explicitly destroyed and
        recreated every time the code is run.        
    */
    $("#chart").remove();
    element.append("<div id='chart' style='margin:auto; text-align:center;'></div>");
    
    
    var dataset = [ 5, 10, 13, 19, 21, 25, 22, 18, 15, 13,
                    11, 12, 15, 20, 18, 17, 16, 18, 23, 25 ];
    var svg = d3.select("#chart").append("svg");
    var w = 800;
    var h = 250;
    var bar_padding = 1;
    // "..." is the spread operator. It unpacks arrays?
    // You can't just call max on the array variable...
    console.log(Math.max(...dataset))
    // Must also compensate for padding in the below.
    var bar_width = w / dataset.length - bar_padding;
    var max_height = h
    var scaled_unit = h / Math.max(...dataset)
    svg.attr({
        "width": w,
        "height": h,
        /*
            The below is an SVG property which forces elements inside to conform to the nearest whole digit.
            Because a half a pixel is rendered as a blur, this forces sharpness.
        */
        "shape-rendering": "crispEdges",
    });
    svg.selectAll("rect").data(dataset).enter().append("rect").attr({
        "fill": "black",
        "width": bar_width,
        "height": function(d) { return d * scaled_unit; },
        "x": function(d, i) { return (bar_width + 1) * i; },
        /* 
            SVG does not allow negative coordinates so we have to "grow from above".
            ...wow this is crude. "Standards" haha damn.
        */
        "y": function(d) { return (Math.max(...dataset) - d) * scaled_unit },
        "fill": function(d) { return "rgb(0, 0, " + (d * 10) + ")"; }
    });
    svg.selectAll("text")
        .data(dataset)
        .enter()
        .append("text")
        .text(function(d) { return d; })
        .attr({
            "text-anchor": "middle",
            "fill": "white",
            "x": function(d, i) { return (bar_width + 1) * i + bar_width / 2 - 1 },
            "y": function(d) { return (Math.max(...dataset) - d) * scaled_unit + 20 },
    });
});

<IPython.core.display.Javascript object>

Scatterplot using scales.

In [54]:
%%javascript
require(['d3'], function(d3){
    /*
        Jupyter notebooks save their state between runtimes
        so the chart needs to be explicitly destroyed and
        recreated every time the code is run.        
    */
    $("#chart").remove();
    element.append("<div id='chart' style='margin:auto; text-align:center;'></div>");
    
    
    var svg = d3.select("#chart").append("svg");
    var w = 800;
    var h = 250;
    var dataset = [
                  [ 5,     20 ],
                  [ 480,   90 ],
                  [ 250,   50 ],
                  [ 100,   33 ],
                  [ 330,   95 ],
                  [ 410,   12 ],
                  [ 475,   44 ],
                  [ 25,    67 ],
                  [ 85,    21 ],
                  [ 220,   88 ]
              ];
    var padding = 20;
    svg.attr({
        "width": w,
        "height": h,
        /*
            The below is an SVG property which forces elements inside to conform to the nearest whole digit.
            Because a half a pixel is rendered as a blur, this forces sharpness.
        */
//         "shape-rendering": "crispEdges",
    });
    var xScale = d3.scale.linear()
                         .domain([0, d3.max(dataset, function(d) { return d[0]; })])
                         .range([padding, w - padding]);
    var yScale = d3.scale.linear()
                         .domain([0, d3.max(dataset, function(d) { return d[1]; })])
                         .range([padding, h - padding]);
    var rScale = d3.scale.linear()
                     .domain([0, d3.max(dataset, function(d) { return d[1]; })])
                     .range([2, 5]);
    svg.selectAll("circ").data(dataset).enter().append("circle").attr({
        "cx": function(d) { return xScale(d[0]); },
        "cy": function(d) { return yScale(d[1]); },
        "fill": "black",
        "r": function(d) { return rScale(d[1]); }
    })
});

<IPython.core.display.Javascript object>

With axis. Axis visual is cleaned up by the CSS in the block below. Moral of the story: groups (g) are very useful for applying styles in SVG.

In [57]:
%%html
<style>
.axis path,
.axis line {
    fill: none;
    stroke: black;
    shape-rendering: crispEdges;
}

.axis text {
    font-family: sans-serif;
    font-size: 11px;
}
</style>

In [58]:
%%javascript
require(['d3'], function(d3){
    /*
        Jupyter notebooks save their state between runtimes
        so the chart needs to be explicitly destroyed and
        recreated every time the code is run.        
    */
    $("#chart").remove();
    element.append("<div id='chart' style='margin:auto; text-align:center;'></div>");
    
    
    var svg = d3.select("#chart").append("svg");
    var w = 800;
    var h = 250;
    var dataset = [
                  [ 5,     20 ],
                  [ 480,   90 ],
                  [ 250,   50 ],
                  [ 100,   33 ],
                  [ 330,   95 ],
                  [ 410,   12 ],
                  [ 475,   44 ],
                  [ 25,    67 ],
                  [ 85,    21 ],
                  [ 220,   88 ]
              ];
    var padding = 20;
    svg.attr({
        "width": w,
        "height": h,
        /*
            The below is an SVG property which forces elements inside to conform to the nearest whole digit.
            Because a half a pixel is rendered as a blur, this forces sharpness.
        */
//         "shape-rendering": "crispEdges",
    });
    var xScale = d3.scale.linear()
                         .domain([0, d3.max(dataset, function(d) { return d[0]; })])
                         .range([padding, w - padding]);
    var yScale = d3.scale.linear()
                         .domain([0, d3.max(dataset, function(d) { return d[1]; })])
                         .range([padding, h - padding]);
    var rScale = d3.scale.linear()
                     .domain([0, d3.max(dataset, function(d) { return d[1]; })])
                     .range([2, 5]);
    svg.selectAll("circ").data(dataset).enter().append("circle").attr({
        "cx": function(d) { return xScale(d[0]); },
        "cy": function(d) { return yScale(d[1]); },
        "fill": "black",
        "r": function(d) { return rScale(d[1]); }
    })
    var xAxis = d3.svg.axis()
                      .scale(xScale)
                      .orient("bottom");
    var yAxis = d3.svg.axis()
                  .scale(yScale)
                  .orient("right");
    svg.append("g").attr("class", "axis").call(xAxis);
    svg.append("g").attr("class", "axis").call(yAxis);
});

<IPython.core.display.Javascript object>